In [1]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                """Alexandru most  dangerous ensemble """
import os
 
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
from sklearn.metrics import r2_score
from __future__ import division
import pandas as pd 
import  random as random
import numpy as np
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
import heapq
from sklearn.linear_model import  LinearRegression,BayesianRidge
from sklearn.ensemble import  RandomForestRegressor,RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.ensemble import *
import math
from sklearn import metrics
from sklearn.tree import *
 
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import ml_metrics
from ml_metrics import quadratic_weighted_kappa
class BadErrorTreshold(Exception):
    pass
 
class EnsembleRegressors(object):
    ''''
    ensembleRegressor is a class of ensembles generated by a  fix number of  iterations.This class is abstract 
    meaning  it  is not possible  to create instances  of it.,,stochasticLearning''  has interface  role. '''
    def __init__(self,regressors,sampling,iterations,test,X,y,k_fold):
        ''''intialize ensemble with  near all  parameters- see that   some of them are  intialized  by other methods'''
        self.sampling=sampling#dropout(cv) ratio-percentage
        if (self.sampling<=0.0 or self.sampling>=1.0):
            raise ValueError('Sampling  must be percent like value between[0,1) ex 0.75, you introduced :',self.sampling)
 
 
        self.test=test#final test set
        self.X=X#original train features
        self.y=y#original train outcome
        self.iterations=iterations#number of  iterations in the  stochastic ensemble
        self.regressors=regressors#ensemble models  for  each iteration      
        self.randomIndexes=[ [] for rounds in range(self.iterations)]#filled in with random dropout  by getStochasticDataSets()
        self.error=[]#error for  each averaged iteration
 
 
        self.stochasticPredictions=[ [] for  rounds in range(self.iterations)]#predictions on random drop outs
        self.finalPredictions=[]
 
        self.x_train=[]#his   next  4 rows are intialized  by getStochasticDataSets() according to random chunks
        self.y_train=[]
        self.x_test=[]
        self.y_test=[]
 
        self.new_x_ts=[]
        self.new_x_tr=[]
        self.new_test_final=[]
        self.k_folds=k_fold
        
    def returnError(self):
        '''returns : error for  each iteration-just like cross-validation'''
        return self.error
    
    def returnRandomIndexes(self):
        '''returns: random  indexes  for  each dropout'''
        return self.randomIndexes
    
    def returnstochasticPredictions(self):
        '''returns:predictions  on random dropous'''
        return self.stochasticPredictions
    
    def getStochasticDataSets(self):
        '''
        For some  potentially  iteration creates a random  idx variable representing the index  for random  dropout.
        Splits the train in  train/test(x_train,y_train,features and outcome for cross val train-x_test,y_test for cross_val test
        .Updated this train-test  variables with corresponding  values.See that  the initial vales are empty in the __init__.
        returns: idx(index of  dropout rows).
        
        '''
        x_train, x_test, y_train, y_test = train_test_split(self.X, self.y, test_size=self.sampling)
        self.x_train=x_train
        self.x_test=x_test
        self.y_train=y_train
        self.y_test=y_test
        #train_i, test_i = train_test_split(np.arange(self.X.shape[0]), train_size = self.sampling)
        #self.x_train=self.X[train_i]
        #self.y_train=self.y[train_i]
        #self.x_test=self.X[test_i]
        #self.y_test=self.y[test_i]
        #idx=test_i
        #return  idx
    
    def training(self,i):
        ''''
        For a  particular iteration ,,i''  calls getStochasticDataSets() method.In this was the randomIndexes feature is
        fiiled with corresponding index dropouts at position ,,i''.Since train/test new data sets according to cross validation
        are   filled  by getStochasticDataSets()  independet  of iteration number , this data sets will update for each new iter.
        performmed in stochasticLearning().'''
        self.getStochasticDataSets()
        #blending- we have x_train,y_train,x_test,y_test   our stochastic  dropouts for diversity
        #we are  going first  to make blend train,blend test from this dropoutsthen will see evaluation on the dropout
        clfs=self.regressors
        n_folds=self.k_folds
        skf=list(StratifiedKFold(self.y_train,n_folds))
        blend_tr=np.zeros((self.x_train.shape[0],len(clfs)))
        blend_ts=np.zeros((self.x_test.shape[0],len(clfs)))#for loca al evaluation
        blend_final_test=np.zeros((self.test.shape[0],len(clfs)))
 
        for  j , clf in  enumerate(clfs):
            print(clf)
            blend_ts_j=np.zeros((self.x_test.shape[0],len(skf)))
            blend_final_j=np.zeros((self.test.shape[0],len(skf)))
            for i ,(tr,ts) in enumerate(skf):
                print("fold",i)
                x_tr=self.x_train[tr]
                y_tr=self.y_train[tr]
                x_ts=self.x_train[ts]
                y_ts=self.y_train[ts]
                clf.fit(x_tr,y_tr)
                y_sub=clf.predict_proba(x_ts)[:,1]
                blend_tr[ts,j]=y_sub
                blend_ts_j[:,i]=clf.predict_proba(self.x_test)[:,1]
                blend_final_j[:,i]=clf.predict_proba(self.test)[:,1]
            blend_ts[:,j]=blend_ts_j.mean(1)
            blend_final_test[:,j]=blend_final_j.mean(1)
        #blend on evaluation
        clf = LogisticRegression()
        clf.fit(blend_tr, self.y_train)
        
        pred_eval=  clf.predict_proba(blend_ts)[:,1]
        self.error.append(log_loss(self.y_test,pred_eval))
        #blend  on final test set:
        final_sub=clf.predict_proba(blend_final_test)[:,1]
        self.finalPredictions.append(final_sub)
        
        
 
        return self
    
    def getFinalPrediction(self,errorTreshold):
       raise NotImplementedError###averages models  according to best top prediction  trehsold  with different implementantions
    
    
        
    
    def stochasticLearning(self):
         raise NotImplementedError#Since EnsemebleRegressors class is abstract this will act  as an interface
                                  #and will be implemented   by  this  EnsembleRegressors sublclasses  for averaging and
                                  #weighted average
    
        
class AveragingModels(EnsembleRegressors):
    '''Inherits everything from EnsembleRegressors  class and implements stochastiLearning method  averaging models 
    for each iteration'''
 
    def stochasticLearning(self):
        
        
        for iter in range(self.iterations):
            print('Stochastic Iteration number ',iter)
            self.training(iter)
 
    def getFinalPrediction(self,errorTreshold):
        if (errorTreshold<=self.iterations and errorTreshold>0):
            
            topErrorIndex=heapq.nsmallest(errorTreshold, range(len(self.error)), self.error.__getitem__)
            finalAvg=0
            for topError in topErrorIndex:
                finalAvg=finalAvg+self.finalPredictions[topError]
            finalAvg=(finalAvg)/float(len(topErrorIndex))
            return finalAvg
        else:
            raise BadErrorTreshold
       
       
    
                                                


In [11]:
#read stuff
import pandas  as pd
import numpy as np
train=pd.read_csv('anotherTrainThing.csv')
test=pd.read_csv('anotherTestThing.csv')
y=train['target']
train=train.drop('target',1)
train=train.drop('ID',1)
ID=test['ID']
 

test=test.drop('ID',1)
from sklearn.metrics import log_loss
trainvec=np.array(train)
testvec=np.array(test)
y=np.array(y)


 

In [4]:
trainvec.shape

(114321, 132)

In [5]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import  xgboost as xgb
"""define classifiers"""
clfs = [RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
            GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=7, n_estimators=100),
            RandomForestClassifier(n_estimators=120, n_jobs=-1, criterion='gini'),
            RandomForestClassifier(n_estimators=80, n_jobs=-1, criterion='entropy'),
            ExtraTreesClassifier(n_estimators=120, n_jobs=-1, criterion='gini'),
            RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
            RandomForestClassifier(n_estimators=150, n_jobs=-1, criterion='entropy'),
            ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
            ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
            GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=7, n_estimators=80),
            ExtraTreesClassifier(n_estimators=70, n_jobs=-1, criterion='gini'),
            GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=50),
            RandomForestClassifier(n_estimators=170, n_jobs=-1, criterion='gini'),
            GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=8, n_estimators=120),
            RandomForestClassifier(n_estimators=180, n_jobs=-1, criterion='entropy'),
            RandomForestClassifier(n_estimators=200, n_jobs=-1, criterion='entropy'),
            ExtraTreesClassifier(n_estimators=120, n_jobs=-1, criterion='gini')
            
            ]


In [22]:
""" run stuff with  0.1 dropout , 3 iterations,predict on testvec, train on trainvec and  y ,do 10
stratified  sampling"""
import  xgboost  as xgb
 
o=AveragingModels(clfs,0.1,1,testvec,trainvec,y,10)
o.stochasticLearning()


Stochastic Iteration number  0
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=10, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9


In [23]:
o.error# please  look  at local error   before  submit

[0.48349370356240173]

In [24]:
# see LB  on 1   iteration
p=o.getFinalPrediction(1)
p=p.reshape(p.shape[0])
prediction=pd.DataFrame({"ID":ID,"PredictedProb":p})
prediction.to_csv("attemp1.csv",index=False)